In [75]:
import pandas as pd
import numpy as np

In [1]:
from transformers import BertTokenizer, BertForPreTraining, BertModel
import torch
my_sentence = "Whatever your sentence is"
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')


c:\users\krystian.kurek\miniconda3\envs\imagesequenceclassification\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Downloading: 100%|██████████████████████████████████████████████████████████████████| 440M/440M [02:59<00:00, 2.45MB/s]
c:\users\krystian.kurek\miniconda3\envs\imagesequenceclassification\lib\site-packages\huggingface_hub\file_download.py:125: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\krystian.kurek\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/ho

In [72]:
def return_embeddings(my_sentence, model=model): 
    input_ids = tokenizer(my_sentence, return_tensors="pt")
    output = model(**input_ids)

    final_layer = output.last_hidden_state
    return final_layer[0][1].detach().numpy().reshape(1, -1)
return_embedings('Whatever').shape

(1, 768)

In [63]:
def process_extracted_keywords_file(filename): 
    df = pd.read_csv(filename)\
        .drop('Unnamed: 0', axis=1)
    df['topic_keywords'] = df['topic_keywords'].apply(eval)
    df = df.explode('topic_keywords')
    df[['topic_keyword', 'probability']] = df[['topic_keywords']].apply(lambda x: [x['topic_keywords'][0], x['topic_keywords'][1]], result_type='expand', axis=1)
   
    df = df.drop('topic_keywords', axis=1)
    return df
bertopic_keywords = process_extracted_keywords_file('../results/bertopic/bertopic_processed_data_2022-11-22_23-14-24_train.csv')

In [64]:
import tqdm

In [65]:
bertopic_keywords.head()

,PMID,topic_number,topic_probs,topic_keyword,probability
0,25763772,52,1.0,pneumonia,0.108995
0,25763772,52,1.0,antimicrobi,0.060756
0,25763772,52,1.0,isol,0.059366
0,25763772,52,1.0,hmkp,0.049176
0,25763772,52,1.0,infect,0.048220


In [94]:
def return_embeddings_for_keywords(keywords, model=model):
    unique_keywords = keywords.drop_duplicates()
    embeddings = []
    for keyword in tqdm.tqdm(unique_keywords['topic_keyword']): 
        embeddings.append(return_embeddings(keyword))
    embeddings = np.concatenate(embeddings, axis=0)
    embeddings = pd.DataFrame(embeddings, columns=list(range(embeddings.shape[1])))
    embeddings['topic_keyword'] = unique_keywords['topic_keyword'].reset_index(drop=True)

    return embeddings

def add_embeddings_to_keywords(df, model=model): 
    keywords_embeddings = return_embeddings_for_keywords(df[['topic_keyword']])
    return df.merge(keywords_embeddings)

bertopic_keywords = add_embeddings_to_keywords(bertopic_keywords, model=model)


100%|████████████████████████████████████████████████████████████████████████████████| 494/494 [00:28<00:00, 17.49it/s]


In [95]:
bertopic_keywords

,PMID,topic_number,topic_probs,topic_keyword,probability,0,1,2,3,4,...,758,759,760,761,762,763,764,765,766,767
0,25763772,52,1.000000,pneumonia,0.108995,-0.194289,-0.255824,-0.134394,-0.148016,0.060968,...,0.801799,-0.046112,0.462496,0.336230,0.147135,-0.449673,-0.159336,0.183655,0.092611,0.061448
1,27334470,52,1.000000,pneumonia,0.108995,-0.194289,-0.255824,-0.134394,-0.148016,0.060968,...,0.801799,-0.046112,0.462496,0.336230,0.147135,-0.449673,-0.159336,0.183655,0.092611,0.061448
2,27346336,52,1.000000,pneumonia,0.108995,-0.194289,-0.255824,-0.134394,-0.148016,0.060968,...,0.801799,-0.046112,0.462496,0.336230,0.147135,-0.449673,-0.159336,0.183655,0.092611,0.061448
3,27641480,52,1.000000,pneumonia,0.108995,-0.194289,-0.255824,-0.134394,-0.148016,0.060968,...,0.801799,-0.046112,0.462496,0.336230,0.147135,-0.449673,-0.159336,0.183655,0.092611,0.061448
4,27798219,52,1.000000,pneumonia,0.108995,-0.194289,-0.255824,-0.134394,-0.148016,0.060968,...,0.801799,-0.046112,0.462496,0.336230,0.147135,-0.449673,-0.159336,0.183655,0.092611,0.061448
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35125,28295933,47,0.901133,gg,0.029372,-0.728101,-0.560432,0.696359,-0.845489,-0.546084,...,0.292864,0.023463,0.890567,-0.099152,-0.290927,-0.135723,-0.090461,0.051818,0.370925,0.745343
35126,28359307,47,1.000000,gg,0.029372,-0.728101,-0.560432,0.696359,-0.845489,-0.546084,...,0.292864,0.023463,0.890567,-0.099152,-0.290927,-0.135723,-0.090461,0.051818,0.370925,0.745343
35127,28470753,47,0.878475,gg,0.029372,-0.728101,-0.560432,0.696359,-0.845489,-0.546084,...,0.292864,0.023463,0.890567,-0.099152,-0.290927,-0.135723,-0.090461,0.051818,0.370925,0.745343
35128,28522288,47,1.000000,gg,0.029372,-0.728101,-0.560432,0.696359,-0.845489,-0.546084,...,0.292864,0.023463,0.890567,-0.099152,-0.290927,-0.135723,-0.090461,0.051818,0.370925,0.745343
